In [5]:
#1. Step - Detect Object

from ultralytics import YOLO

# Load a model
model = YOLO('yolov5nu.pt')  # load an official model

# Predict with the model
results = model(r'C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019\camera_front\000001.jpg')  # predict on an image
import torch
from pathlib import Path
from tqdm import tqdm
import numpy as np
import cv2
import json

# Define paths
data_dir = Path(r'C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019')
images_dir = data_dir / (r'C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019\camera_front')
labels_dir = data_dir / (r'C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019\groundtruth_obj3d')
# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Create a directory to save the bounding box coordinates
output_dir = data_dir / "Step-1-output"
output_dir.mkdir(exist_ok=True)

# Process each image in the dataset
for image_path in tqdm(list(images_dir.glob("*.jpg"))):
    # Perform inference
    results = model(image_path)
    
    # Extract bounding box coordinates for vehicles
    bboxes = results.xyxy[0].cpu().numpy()  # Get bounding boxes in (x_min, y_min, x_max, y_max, confidence) format
    #vehicle_bboxes = bboxes[bboxes[:, 5] == 2]  # Assuming class 2 corresponds to vehicles
    # Extract bounding box coordinates for vehicles with classes 2, 5, and 7
    vehicle_bboxes = bboxes[np.isin(bboxes[:, 5], [2, 5, 7])]

    # Save bounding box coordinates
    image_name = image_path.stem
    np.savetxt(output_dir / f"{image_name}_vehicle_coordinates.txt", vehicle_bboxes[:, :4], fmt='%d')

   
    
    # Optionally, visualize the detection results
    img = cv2.imread(str(image_path))
    for bbox in vehicle_bboxes:
        xmin, ymin, xmax, ymax = bbox[:4].astype(int)
        cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    cv2.imwrite(str(output_dir / f"{image_name}_vehicle_detection.jpg"), img)


image 1/1 C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019\camera_front\000001.jpg: 224x640 23 cars, 128.1ms
Speed: 15.4ms preprocess, 128.1ms inference, 1719.5ms postprocess per image at shape (1, 3, 224, 640)


Using cache found in C:\Users\rahma/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-5-11 Python-3.12.3 torch-2.3.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
100%|██████████████████████████████████████████████████████████████████████████████████| 535/535 [01:16<00:00,  7.00it/s]


In [4]:
#2. Step - Project Point Cloud
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import json

class calib_astyx():
    """This class is designed to manage the calibration of sensor data in autonomous vehicle contexts.
    
    It supports the conversion of sensor data between different coordinate systems, specifically for radar,
    lidar, and camera sensors, to a common reference coordinate system and vice versa. It also handles the
    projection of points from the reference coordinate system onto the camera image plane.
    """
    
    def __init__(self, file):
        """Initialize the calibration object by loading sensor calibration data from a JSON file.
        
        Parameters:
        - file: A string path to the JSON file containing calibration data for radar, lidar, and camera sensors.
        
        The method extracts transformation matrices for each sensor to the reference coordinate system and
        the intrinsic camera matrix.
        """

        # Load calibration data from a JSON file
        with open(file) as json_file:
            data = json.load(json_file)
            
        # Calibration matrices for converting radar, lidar, and camera data to a reference coordinate system
        self.radar2ref = np.array(data["sensors"][0]["calib_data"]["T_to_ref_COS"])  # Radar to reference
        self.lidar2ref_cos = np.array(data["sensors"][1]["calib_data"]["T_to_ref_COS"])  # Lidar to reference (COS means Coordinate System)
        self.camera2ref = np.array(data["sensors"][2]["calib_data"]["T_to_ref_COS"])  # Camera to reference
        self.K = np.array(data["sensors"][2]["calib_data"]["K"])  # Intrinsic camera matrix
        
        # Compute inverse transformations for mapping from the reference coordinate system back to sensor-specific coordinate systems
        self.ref2radar = self.inv_trans(self.radar2ref)
        self.ref2lidar = self.inv_trans(self.lidar2ref_cos)
        self.ref2camera = self.inv_trans(self.camera2ref)

    @staticmethod
    def inv_trans(T):
        """Compute the inverse transformation matrix for a given sensor to reference coordinate transformation.
        
        Parameters:
        - T: A numpy array representing the transformation matrix from sensor to reference coordinates.
        
        Returns:
        - The inverse transformation matrix as a numpy array, which can be used to map points from the
          reference coordinate system back to the sensor-specific coordinate system.
        """
        rotation = np.linalg.inv(T[0:3, 0:3])  # Invert the rotation part
        translation = T[0:3, 3]
        translation = -1 * np.dot(rotation, translation.T)  # Invert the translation part
        translation = np.reshape(translation, (3, 1))
        Q = np.hstack((rotation, translation))  # Reassemble the inverted transformation matrix

        return Q
    
    def lidar2ref(self, points):
        """Convert lidar points from the lidar coordinate system to the reference coordinate system.
        
        Parameters:
        - points: A numpy array of points in the lidar coordinate system.
        
        Returns:
        - A numpy array of the same points transformed to the reference coordinate system.
        """
        n = points.shape[0]
        
        points_hom = np.hstack((points, np.ones((n,1))))  # Convert points to homogeneous coordinates
        points_ref = np.dot(points_hom, np.transpose(self.lidar2ref_cos))  # Transform points to reference coordinate system
        
        return points_ref[:,0:3]  # Return the transformed points, discarding the homogeneous coordinate


    def ref2Camera(self, points, img_size):
        """Project points from the reference coordinate system onto the camera image plane.
        
        Parameters:
        - points: A numpy array of points in the reference coordinate system.
        - img_size: A tuple representing the size of the camera image (width, height).
        
        Returns:
        - A tuple containing a numpy array of the projected points on the camera image plane and a mask
          array indicating which points are within the image frame.
        """
        obj_image = np.dot(self.ref2camera[0:3, 0:3], points.T)  # Apply rotation
        T = self.ref2camera[0:3, 3]
        obj_image = obj_image + T[:, np.newaxis]  # Apply translation
        obj_image = np.dot(self.K, obj_image)  # Apply intrinsic camera matrix
        obj_image = obj_image / obj_image[2]  # Normalize by the third row to project onto image plane
        obj_image = np.delete(obj_image, 2, 0)  # Remove the third row
        
        # Create a mask to filter out points that are outside the image frame or behind the camera
        mask = (obj_image[0,:] <= img_size[0]) & \
               (obj_image[1,:] <= img_size[1]) & \
               (obj_image[0,:] >= 0) & (obj_image[1,:] >= 0) & \
               (points[:,0] >= 0)
        return obj_image[:, mask], mask  # Return the projected points and the mask

# Define data directories
data_dir = Path(r'C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019')
images_dir = data_dir / "camera_front"
output_dir = data_dir / "step-2-output"
output_dir.mkdir(exist_ok=True)

# Process each image in the dataset
for image_path in tqdm(list(images_dir.glob("*.jpg"))):
    img = cv2.imread(str(image_path))
    lidar = np.loadtxt(data_dir / "lidar_vlp16" / f"{image_path.stem}.txt", skiprows=1)
    radar = np.loadtxt(data_dir / "radar_6455" / f"{image_path.stem}.txt", skiprows=2)
    calib = calib_astyx(data_dir / "calibration" / f"{image_path.stem}.json")

    # Transform lidar points to the reference coordinate system and then project onto the camera image
    lidar[:, 0:3] = calib.lidar2ref(lidar[:, 0:3])
    lidar_on_img, lidar_on_img_mask = calib.ref2Camera(lidar[:, 0:3], (img.shape[1], img.shape[0]))

    # Transform radar points to the reference coordinate system and then project onto the camera image
    radar_on_img, radar_on_img_mask = calib.ref2Camera(radar[:, 0:3], (img.shape[1], img.shape[0]))

    # Plot and save the lidar and radar points on the camera image
    plt.figure(figsize=(30, 30))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.scatter(lidar_on_img[0, :], lidar_on_img[1, :], c=lidar[lidar_on_img_mask, 3], s=100)
    plt.axis("off")
    plt.savefig(str(output_dir / f"{image_path.stem}_lidar_on_img.jpg"))
    plt.close()

    plt.figure(figsize=(30, 30))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.scatter(radar_on_img[0, :], radar_on_img[1, :], c=radar[radar_on_img_mask, 3], s=100)
    plt.axis("off")
    plt.savefig(str(output_dir / f"{image_path.stem}_radar_on_img.jpg"))
    plt.close()


100%|██████████████████████████████████████████████████████████████████████████████████| 546/546 [08:12<00:00,  1.11it/s]


In [26]:
#3. Step - Filter Points
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import json

class calib_astyx():
    """This class is designed to manage the calibration of sensor data in autonomous vehicle contexts.
    
    It supports the conversion of sensor data between different coordinate systems, specifically for radar,
    lidar, and camera sensors, to a common reference coordinate system and vice versa. It also handles the
    projection of points from the reference coordinate system onto the camera image plane.
    """
    
    def __init__(self, file):
        """Initialize the calibration object by loading sensor calibration data from a JSON file.
        
        Parameters:
        - file: A string path to the JSON file containing calibration data for radar, lidar, and camera sensors.
        
        The method extracts transformation matrices for each sensor to the reference coordinate system and
        the intrinsic camera matrix.
        """

        # Load calibration data from a JSON file
        with open(file) as json_file:
            data = json.load(json_file)
            
        # Calibration matrices for converting radar, lidar, and camera data to a reference coordinate system
        self.radar2ref = np.array(data["sensors"][0]["calib_data"]["T_to_ref_COS"])  # Radar to reference
        self.lidar2ref_cos = np.array(data["sensors"][1]["calib_data"]["T_to_ref_COS"])  # Lidar to reference (COS means Coordinate System)
        self.camera2ref = np.array(data["sensors"][2]["calib_data"]["T_to_ref_COS"])  # Camera to reference
        self.K = np.array(data["sensors"][2]["calib_data"]["K"])  # Intrinsic camera matrix
        
        # Compute inverse transformations for mapping from the reference coordinate system back to sensor-specific coordinate systems
        self.ref2radar = self.inv_trans(self.radar2ref)
        self.ref2lidar = self.inv_trans(self.lidar2ref_cos)
        self.ref2camera = self.inv_trans(self.camera2ref)

    @staticmethod
    def inv_trans(T):
        """Compute the inverse transformation matrix for a given sensor to reference coordinate transformation.
        
        Parameters:
        - T: A numpy array representing the transformation matrix from sensor to reference coordinates.
        
        Returns:
        - The inverse transformation matrix as a numpy array, which can be used to map points from the
          reference coordinate system back to the sensor-specific coordinate system.
        """
        rotation = np.linalg.inv(T[0:3, 0:3])  # Invert the rotation part
        translation = T[0:3, 3]
        translation = -1 * np.dot(rotation, translation.T)  # Invert the translation part
        translation = np.reshape(translation, (3, 1))
        Q = np.hstack((rotation, translation))  # Reassemble the inverted transformation matrix

        return Q
    
    def lidar2ref(self, points):
        """Convert lidar points from the lidar coordinate system to the reference coordinate system.
        
        Parameters:
        - points: A numpy array of points in the lidar coordinate system.
        
        Returns:
        - A numpy array of the same points transformed to the reference coordinate system.
        """
        n = points.shape[0]
        
        points_hom = np.hstack((points, np.ones((n,1))))  # Convert points to homogeneous coordinates
        points_ref = np.dot(points_hom, np.transpose(self.lidar2ref_cos))  # Transform points to reference coordinate system
        
        return points_ref[:,0:3]  # Return the transformed points, discarding the homogeneous coordinate


    def ref2Camera(self, points, img_size, bbox_list):
        """Project points from the reference coordinate system onto the camera image plane.
        
        Parameters:
        - points: A numpy array of points in the reference coordinate system.
        - img_size: A tuple representing the size of the camera image (width, height).
        - bbox_list: A list of tuples, each tuple representing the bounding box coordinates (x_min, y_min, x_max, y_max).
        
        Returns:
        - A tuple containing a numpy array of the projected points on the camera image plane and a mask
          array indicating which points are within the image frame and inside the bounding box.
        """
        all_points = np.array([])
        all_mask = np.array([], dtype=bool)
        for bbox in bbox_list:
            obj_image = np.dot(self.ref2camera[0:3, 0:3], points.T)  # Apply rotation
            T = self.ref2camera[0:3, 3]
            obj_image = obj_image + T[:, np.newaxis]  # Apply translation
            obj_image = np.dot(self.K, obj_image)  # Apply intrinsic camera matrix
            obj_image = obj_image / obj_image[2]  # Normalize by the third row to project onto image plane
            obj_image = np.delete(obj_image, 2, 0)  # Remove the third row

            # Create a mask to filter out points that are outside the image frame or outside the bounding box
            mask = (obj_image[0, :] <= img_size[0]) & \
                   (obj_image[1, :] <= img_size[1]) & \
                   (obj_image[0, :] >= 0) & (obj_image[1, :] >= 0) & \
                   (obj_image[0, :] >= bbox[0]) & (obj_image[0, :] <= bbox[2]) & \
                   (obj_image[1, :] >= bbox[1]) & (obj_image[1, :] <= bbox[3])

            if len(all_points) == 0:
                all_points = obj_image[:, mask].T
            else:
                all_points = np.concatenate((all_points, obj_image[:, mask].T), axis=0)
            all_mask = np.concatenate((all_mask, mask), axis=0)

        return all_points, all_mask  # Return the projected points and the mask


def load_bbox_from_file(file_path):
    """Load bounding box coordinates from a text file.

    Parameters:
    - file_path: A string representing the path to the text file containing bounding box coordinates.

    Returns:
    - A list of tuples, each tuple representing the bounding box coordinates (x_min, y_min, x_max, y_max).
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()
        bbox_list = []
        for line in lines:
            bbox = tuple(map(int, line.strip().split()))
            bbox_list.append(bbox)
    return bbox_list


# Define data directories
data_dir = Path(r'C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019')
images_dir = data_dir / "camera_front"
output_dir = data_dir / "step-3-output"
output_dir.mkdir(exist_ok=True)

# Process each image in the dataset
for image_path in tqdm(list(images_dir.glob("*.jpg"))):
    img = cv2.imread(str(image_path))
    lidar = np.loadtxt(data_dir / "lidar_vlp16" / f"{image_path.stem}.txt", skiprows=1)
    radar = np.loadtxt(data_dir / "radar_6455" / f"{image_path.stem}.txt", skiprows=2)
    calib = calib_astyx(data_dir / "calibration" / f"{image_path.stem}.json")

    # Load bounding box coordinates for the current image
    bbox_list = load_bbox_from_file(data_dir / "Step-1-output" / f"{image_path.stem}_vehicle_coordinates.txt")

    # Transform lidar points to the reference coordinate system
    lidar[:, 0:3] = calib.lidar2ref(lidar[:, 0:3])

    # Filter lidar points outside bounding boxes and project onto the camera image
    lidar_on_img, lidar_on_img_mask = calib.ref2Camera(lidar[:, 0:3], (img.shape[1], img.shape[0]), bbox_list)

    # Transform radar points to the reference coordinate system
    radar[:, 0:3] = calib.lidar2ref(radar[:, 0:3])

    # Filter radar points outside bounding boxes and project onto the camera image
    radar_on_img, radar_on_img_mask = calib.ref2Camera(radar[:, 0:3], (img.shape[1], img.shape[0]), bbox_list)

    # Plot and save the filtered lidar and radar points on the camera image
    plt.figure(figsize=(30, 30))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.scatter(lidar_on_img[:, 0], lidar_on_img[:, 1], c='b', s=100)
    plt.axis("off")
    plt.savefig(str(output_dir / f"{image_path.stem}_lidar_on_img.jpg"))
    plt.close()

    plt.figure(figsize=(30, 30))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.scatter(radar_on_img[:, 0], radar_on_img[:, 1], c='r', s=100)
    plt.axis("off")
    plt.savefig(str(output_dir / f"{image_path.stem}_radar_on_img.jpg"))
    plt.close()


100%|██████████████████████████████████████████████████████████████████████████████████| 535/535 [08:06<00:00,  1.10it/s]


In [1]:
#4. Step - Distance Estimation
import json
import matplotlib.pyplot as plt
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm

class calib_astyx():
    """This class is designed to manage the calibration of sensor data in autonomous vehicle contexts.
    
    It supports the conversion of sensor data between different coordinate systems, specifically for radar,
    lidar, and camera sensors, to a common reference coordinate system and vice versa. It also handles the
    projection of points from the reference coordinate system onto the camera image plane.
    """
    
    def __init__(self, file):
        """Initialize the calibration object by loading sensor calibration data from a JSON file.
        
        Parameters:
        - file: A string path to the JSON file containing calibration data for radar, lidar, and camera sensors.
        
        The method extracts transformation matrices for each sensor to the reference coordinate system and
        the intrinsic camera matrix.
        """

        # Load calibration data from a JSON file
        with open(file) as json_file:
            data = json.load(json_file)
            
        # Calibration matrices for converting radar, lidar, and camera data to a reference coordinate system
        self.radar2ref = np.array(data["sensors"][0]["calib_data"]["T_to_ref_COS"])  # Radar to reference
        self.lidar2ref_cos = np.array(data["sensors"][1]["calib_data"]["T_to_ref_COS"])  # Lidar to reference (COS means Coordinate System)
        self.camera2ref = np.array(data["sensors"][2]["calib_data"]["T_to_ref_COS"])  # Camera to reference
        self.K = np.array(data["sensors"][2]["calib_data"]["K"])  # Intrinsic camera matrix
        
        # Compute inverse transformations for mapping from the reference coordinate system back to sensor-specific coordinate systems
        self.ref2radar = self.inv_trans(self.radar2ref)
        self.ref2lidar = self.inv_trans(self.lidar2ref_cos)
        self.ref2camera = self.inv_trans(self.camera2ref)

    @staticmethod
    def inv_trans(T):
        """Compute the inverse transformation matrix for a given sensor to reference coordinate transformation.
        
        Parameters:
        - T: A numpy array representing the transformation matrix from sensor to reference coordinates.
        
        Returns:
        - The inverse transformation matrix as a numpy array, which can be used to map points from the
          reference coordinate system back to the sensor-specific coordinate system.
        """
        rotation = np.linalg.inv(T[0:3, 0:3])  # Invert the rotation part
        translation = T[0:3, 3]
        translation = -1 * np.dot(rotation, translation.T)  # Invert the translation part
        translation = np.reshape(translation, (3, 1))
        Q = np.hstack((rotation, translation))  # Reassemble the inverted transformation matrix

        return Q
    
    def lidar2ref(self, points):
        """Convert lidar points from the lidar coordinate system to the reference coordinate system.
        
        Parameters:
        - points: A numpy array of points in the lidar coordinate system.
        
        Returns:
        - A numpy array of the same points transformed to the reference coordinate system.
        """
        n = points.shape[0]
        
        points_hom = np.hstack((points, np.ones((n,1))))  # Convert points to homogeneous coordinates
        points_ref = np.dot(points_hom, np.transpose(self.lidar2ref_cos))  # Transform points to reference coordinate system
        
        return points_ref[:,0:3]  # Return the transformed points, discarding the homogeneous coordinate


    def ref2Camera(self, points, img_size, bbox_list):
        """Project points from the reference coordinate system onto the camera image plane.
        
        Parameters:
        - points: A numpy array of points in the reference coordinate system.
        - img_size: A tuple representing the size of the camera image (width, height).
        - bbox_list: A list of tuples representing the bounding box coordinates [(x_min, y_min, x_max, y_max), ...].
        
        Returns:
        - A tuple containing a numpy array of the projected points on the camera image plane and a mask
          array indicating which points are within the image frame and inside the bounding box.
        """
        obj_image_list = []
        mask_list = []
        
        for bbox in bbox_list:
            obj_image = np.dot(self.ref2camera[0:3, 0:3], points.T)  # Apply rotation
            T = self.ref2camera[0:3, 3]
            obj_image = obj_image + T[:, np.newaxis]  # Apply translation
            obj_image = np.dot(self.K, obj_image)  # Apply intrinsic camera matrix
            obj_image = obj_image / obj_image[2]  # Normalize by the third row to project onto image plane
            obj_image = np.delete(obj_image, 2, 0)  # Remove the third row

            # Create a mask to filter out points that are outside the image frame or outside the bounding box
            mask = (obj_image[0, :] <= img_size[0]) & \
                   (obj_image[1, :] <= img_size[1]) & \
                   (obj_image[0, :] >= 0) & (obj_image[1, :] >= 0) & \
                   (obj_image[0, :] >= bbox[0]) & (obj_image[0, :] <= bbox[2]) & \
                   (obj_image[1, :] >= bbox[1]) & (obj_image[1, :] <= bbox[3])

            obj_image_list.append(obj_image[:, mask])
            mask_list.append(mask)
        
        return obj_image_list, mask_list  # Return the projected points list and the mask list
    
    def mean_distance(self, points, mask):
        """Calculate the mean Euclidean distance of the points from the camera's viewpoint inside the bounding box.
        
        Parameters:
        - points: A numpy array of points in 3D space.
        - mask: A boolean mask indicating which points are inside the bounding box.
        
        Returns:
        - The mean Euclidean distance of the points from the camera's viewpoint inside the bounding box.
        """
        camera_points = np.dot(self.ref2camera[0:3, 0:3], points) + self.ref2camera[0:3, 3][:, np.newaxis]
        distance = np.linalg.norm(camera_points, axis=0)
        return np.mean(distance)

# Define the root directory and the output directory
root_dir = Path(r'C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019')
output_dir = root_dir / "step-4-output"

# Ensure the output directory exists, if not, create it
output_dir.mkdir(parents=True, exist_ok=True)

# Iterate through all images
images_dir = root_dir / "Step-1-output"
for image_path in tqdm(list(images_dir.glob("*.jpg"))):
    # Load image, lidar, radar, and calibration data
    img = cv2.imread(str(image_path))
    i = int(image_path.stem.split("_")[0])  # Extract the image index from the file name

    lidar_path = root_dir / "lidar_vlp16" / f"{i:06d}.txt"
    radar_path = root_dir / "radar_6455" / f"{i:06d}.txt"
    calib_path = root_dir / "calibration" / f"{i:06d}.json"

    lidar = np.loadtxt(str(lidar_path), skiprows=1)
    radar = np.loadtxt(str(radar_path), skiprows=2)
    calib = calib_astyx(str(calib_path))

    # Load bounding box coordinates from file
    bbox_file = root_dir / "Step-1-output" / f"{i:06d}_vehicle_coordinates.txt"
    bounding_boxes = []
    with open(bbox_file, 'r') as f:
        for line in f:
            bbox_data = line.strip().split()  # Split by spaces
            bbox = tuple(map(int, bbox_data))  # Convert to tuple of integers
            bounding_boxes.append(bbox)

    # Process Lidar data
    lidar[:, 0:3] = calib.lidar2ref(lidar[:, 0:3])
    lidar_on_img, lidar_on_img_mask = calib.ref2Camera(lidar[:, 0:3], (img.shape[1], img.shape[0]), bounding_boxes)

    # Process Radar data
    radar_on_img, radar_on_img_mask = calib.ref2Camera(radar[:, 0:3], (img.shape[1], img.shape[0]), bounding_boxes)

    # Calculate mean distances for Lidar and Radar
    lidar_points_inside_bbox = [lidar[mask] for mask in lidar_on_img_mask]
    lidar_mean_distance = [calib.mean_distance(points[:, 0:3].T, np.ones(len(points), dtype=bool)) for points in lidar_points_inside_bbox]

    radar_points_inside_bbox = [radar[mask] for mask in radar_on_img_mask]
    radar_mean_distance = [calib.mean_distance(points[:, 0:3].T, np.ones(len(points), dtype=bool)) for points in radar_points_inside_bbox]

    # Plot and save the visualization
    plt.figure(figsize=(30, 30))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    for points, lidar_distance, radar_distance, bbox in zip(lidar_on_img, lidar_mean_distance, radar_mean_distance, bounding_boxes):
        text_x = (bbox[0] + bbox[2]) / 2
        text_y = bbox[1] - 10  # Move the text slightly above the bounding box
        plt.text(text_x, text_y, f' {lidar_distance:.2f},{radar_distance:.2f}', color='white', fontsize=16, ha='center')
    plt.axis("off")

    # Save the output image
    output_image_path = output_dir / f"{image_path.stem}_lidar_radar_on_img.jpg"
    plt.savefig(str(output_image_path))
    plt.close()

print("Processing completed.")


  0%|                                                                                            | 0/546 [00:00<?, ?it/s]C:\Users\rahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\rahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
  0%|▏                                                                                   | 1/546 [00:00<06:00,  1.51it/s]C:\Users\rahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\rahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dty

Processing completed.


In [ ]:
#5. Step - Data Fusion
import os
import json
import matplotlib.pyplot as plt
import cv2
import numpy as np

class calib_astyx():
    """This class is designed to manage the calibration of sensor data in autonomous vehicle contexts.
    
    It supports the conversion of sensor data between different coordinate systems, specifically for radar,
    lidar, and camera sensors, to a common reference coordinate system and vice versa. It also handles the
    projection of points from the reference coordinate system onto the camera image plane.
    """
    
    def __init__(self, file):
        """Initialize the calibration object by loading sensor calibration data from a JSON file.
        
        Parameters:
        - file: A string path to the JSON file containing calibration data for radar, lidar, and camera sensors.
        
        The method extracts transformation matrices for each sensor to the reference coordinate system and
        the intrinsic camera matrix.
        """

        # Load calibration data from a JSON file
        with open(file) as json_file:
            data = json.load(json_file)
            
        # Calibration matrices for converting radar, lidar, and camera data to a reference coordinate system
        self.radar2ref = np.array(data["sensors"][0]["calib_data"]["T_to_ref_COS"])  # Radar to reference
        self.lidar2ref_cos = np.array(data["sensors"][1]["calib_data"]["T_to_ref_COS"])  # Lidar to reference (COS means Coordinate System)
        self.camera2ref = np.array(data["sensors"][2]["calib_data"]["T_to_ref_COS"])  # Camera to reference
        self.K = np.array(data["sensors"][2]["calib_data"]["K"])  # Intrinsic camera matrix
        
        # Compute inverse transformations for mapping from the reference coordinate system back to sensor-specific coordinate systems
        self.ref2radar = self.inv_trans(self.radar2ref)
        self.ref2lidar = self.inv_trans(self.lidar2ref_cos)
        self.ref2camera = self.inv_trans(self.camera2ref)

    @staticmethod
    def inv_trans(T):
        """Compute the inverse transformation matrix for a given sensor to reference coordinate transformation.
        
        Parameters:
        - T: A numpy array representing the transformation matrix from sensor to reference coordinates.
        
        Returns:
        - The inverse transformation matrix as a numpy array, which can be used to map points from the
          reference coordinate system back to the sensor-specific coordinate system.
        """
        rotation = np.linalg.inv(T[0:3, 0:3])  # Invert the rotation part
        translation = T[0:3, 3]
        translation = -1 * np.dot(rotation, translation.T)  # Invert the translation part
        translation = np.reshape(translation, (3, 1))
        Q = np.hstack((rotation, translation))  # Reassemble the inverted transformation matrix

        return Q
    
    def lidar2ref(self, points):
        """Convert lidar points from the lidar coordinate system to the reference coordinate system.
        
        Parameters:
        - points: A numpy array of points in the lidar coordinate system.
        
        Returns:
        - A numpy array of the same points transformed to the reference coordinate system.
        """
        n = points.shape[0]
        
        points_hom = np.hstack((points, np.ones((n,1))))  # Convert points to homogeneous coordinates
        points_ref = np.dot(points_hom, np.transpose(self.lidar2ref_cos))  # Transform points to reference coordinate system
        
        return points_ref[:,0:3]  # Return the transformed points, discarding the homogeneous coordinate


    def ref2Camera(self, points, img_size, bbox_list):
        """Project points from the reference coordinate system onto the camera image plane.
        
        Parameters:
        - points: A numpy array of points in the reference coordinate system.
        - img_size: A tuple representing the size of the camera image (width, height).
        - bbox_list: A list of tuples representing the bounding box coordinates [(x_min, y_min, x_max, y_max), ...].
        
        Returns:
        - A tuple containing a numpy array of the projected points on the camera image plane and a mask
          array indicating which points are within the image frame and inside the bounding box.
        """
        obj_image_list = []
        mask_list = []
        
        for bbox in bbox_list:
            obj_image = np.dot(self.ref2camera[0:3, 0:3], points.T)  # Apply rotation
            T = self.ref2camera[0:3, 3]
            obj_image = obj_image + T[:, np.newaxis]  # Apply translation
            obj_image = np.dot(self.K, obj_image)  # Apply intrinsic camera matrix
            obj_image = obj_image / obj_image[2]  # Normalize by the third row to project onto image plane
            obj_image = np.delete(obj_image, 2, 0)  # Remove the third row

            # Create a mask to filter out points that are outside the image frame or outside the bounding box
            mask = (obj_image[0, :] <= img_size[0]) & \
                   (obj_image[1, :] <= img_size[1]) & \
                   (obj_image[0, :] >= 0) & (obj_image[1, :] >= 0) & \
                   (obj_image[0, :] >= bbox[0]) & (obj_image[0, :] <= bbox[2]) & \
                   (obj_image[1, :] >= bbox[1]) & (obj_image[1, :] <= bbox[3])

            obj_image_list.append(obj_image[:, mask])
            mask_list.append(mask)
        
        return obj_image_list, mask_list  # Return the projected points list and the mask list
    
    def mean_distance(self, points, mask):
        """Calculate the mean Euclidean distance of the points from the camera's viewpoint inside the bounding box.
        
        Parameters:
        - points: A numpy array of points in 3D space.
        - mask: A boolean mask indicating which points are inside the bounding box.
        
        Returns:
        - The mean Euclidean distance of the points from the camera's viewpoint inside the bounding box.
        """
        camera_points = np.dot(self.ref2camera[0:3, 0:3], points) + self.ref2camera[0:3, 3][:, np.newaxis]
        distance = np.linalg.norm(camera_points, axis=0)
        return np.mean(distance)

# Function to calculate fused distance using weighted averaging
def calculate_fused_distance(lidar_mean_distance, radar_mean_distance):
    lidar_weight = 0.7  # Weight for lidar distance
    radar_weight = 0.3  # Weight for radar distance
    return lidar_weight * lidar_mean_distance + radar_weight * radar_mean_distance

# Define the directory paths
input_dir = r'C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019\Step-1-output'
lidar_dir = r'C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019\lidar_vlp16'
radar_dir = r'C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019\radar_6455'
calibration_dir = r'C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019\calibration'
output_dir = r'C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019\step-5-output'

# Loop over all images in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        # Extract the image index from the filename
        image_index = filename.split('_')[0]

        # Read the image
        img = cv2.imread(os.path.join(input_dir, filename))

        # Load lidar and radar data
        lidar_data = np.loadtxt(os.path.join(lidar_dir, f"{image_index}.txt"), skiprows=1)
        radar_data = np.loadtxt(os.path.join(radar_dir, f"{image_index}.txt"), skiprows=2)

        # Load calibration data
        calib = calib_astyx(os.path.join(calibration_dir, f"{image_index}.json"))

        # Load bounding box coordinates from file
        bbox_file = os.path.join(input_dir, f"{image_index}_vehicle_coordinates.txt")
        bounding_boxes = []
        with open(bbox_file, 'r') as f:
            for line in f:
                bbox_data = line.strip().split()  # Split by spaces
                bbox = tuple(map(int, bbox_data))  # Convert to tuple of integers
                bounding_boxes.append(bbox)

        # Convert lidar points to reference coordinate system
        lidar_data[:, 0:3] = calib.lidar2ref(lidar_data[:, 0:3])

        # Project lidar and radar points onto the camera image plane
        lidarOnImg, lidarOnImgMask = calib.ref2Camera(lidar_data[:, 0:3], (img.shape[1], img.shape[0]), bounding_boxes)
        radarOnImg, radarOnImgMask = calib.ref2Camera(radar_data[:, 0:3], (img.shape[1], img.shape[0]), bounding_boxes)

        # Calculate mean distances
        lidar_points_inside_bbox = [lidar_data[mask] for mask in lidarOnImgMask]
        lidar_mean_distance = [calib.mean_distance(points[:, 0:3].T, np.ones(len(points), dtype=bool)) for points in lidar_points_inside_bbox]

        radar_points_inside_bbox = [radar_data[mask] for mask in radarOnImgMask]
        radar_mean_distance = [calib.mean_distance(points[:, 0:3].T, np.ones(len(points), dtype=bool)) for points in radar_points_inside_bbox]

        # Calculate fused distances and annotate image
        plt.figure(figsize=(30, 30))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        for points, lidar_distance, radar_distance, bbox in zip(lidarOnImg, lidar_mean_distance, radar_mean_distance, bounding_boxes):
            fused_distance = calculate_fused_distance(lidar_distance, radar_distance)
            text_x = (bbox[0] + bbox[2]) / 2
            text_y = bbox[1] - 10  # Move the text slightly above the bounding box
            plt.text(text_x, text_y, f' {fused_distance:.2f}', color='red', fontsize=16, ha='center')
        plt.axis("off")

        # Save annotated image
        output_filename = os.path.join(output_dir, f"{image_index}_vehicle_detection_annotated.jpg")
        plt.savefig(output_filename)
        plt.close()


C:\Users\rahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\rahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\rahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\rahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\rahma\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\rahma\AppData\Loca

In [4]:
#6. Step - Evaluation
import os
import json
import matplotlib.pyplot as plt
import cv2
import numpy as np

class calib_astyx():
    """This class is designed to manage the calibration of sensor data in autonomous vehicle contexts.
    
    It supports the conversion of sensor data between different coordinate systems, specifically for radar,
    lidar, and camera sensors, to a common reference coordinate system and vice versa. It also handles the
    projection of points from the reference coordinate system onto the camera image plane.
    """
    
    def __init__(self, file):
        """Initialize the calibration object by loading sensor calibration data from a JSON file.
        
        Parameters:
        - file: A string path to the JSON file containing calibration data for radar, lidar, and camera sensors.
        
        The method extracts transformation matrices for each sensor to the reference coordinate system and
        the intrinsic camera matrix.
        """

        # Load calibration data from a JSON file
        with open(file) as json_file:
            data = json.load(json_file)
            
        # Calibration matrices for converting radar, lidar, and camera data to a reference coordinate system
        self.radar2ref = np.array(data["sensors"][0]["calib_data"]["T_to_ref_COS"])  # Radar to reference
        self.lidar2ref_cos = np.array(data["sensors"][1]["calib_data"]["T_to_ref_COS"])  # Lidar to reference (COS means Coordinate System)
        self.camera2ref = np.array(data["sensors"][2]["calib_data"]["T_to_ref_COS"])  # Camera to reference
        self.K = np.array(data["sensors"][2]["calib_data"]["K"])  # Intrinsic camera matrix
        
        # Compute inverse transformations for mapping from the reference coordinate system back to sensor-specific coordinate systems
        self.ref2radar = self.inv_trans(self.radar2ref)
        self.ref2lidar = self.inv_trans(self.lidar2ref_cos)
        self.ref2camera = self.inv_trans(self.camera2ref)

    @staticmethod
    def inv_trans(T):
        """Compute the inverse transformation matrix for a given sensor to reference coordinate transformation.
        
        Parameters:
        - T: A numpy array representing the transformation matrix from sensor to reference coordinates.
        
        Returns:
        - The inverse transformation matrix as a numpy array, which can be used to map points from the
          reference coordinate system back to the sensor-specific coordinate system.
        """
        rotation = np.linalg.inv(T[0:3, 0:3])  # Invert the rotation part
        translation = T[0:3, 3]
        translation = -1 * np.dot(rotation, translation.T)  # Invert the translation part
        translation = np.reshape(translation, (3, 1))
        Q = np.hstack((rotation, translation))  # Reassemble the inverted transformation matrix

        return Q
    
    def lidar2ref(self, points):
        """Convert lidar points from the lidar coordinate system to the reference coordinate system.
        
        Parameters:
        - points: A numpy array of points in the lidar coordinate system.
        
        Returns:
        - A numpy array of the same points transformed to the reference coordinate system.
        """
        n = points.shape[0]
        
        points_hom = np.hstack((points, np.ones((n,1))))  # Convert points to homogeneous coordinates
        points_ref = np.dot(points_hom, np.transpose(self.lidar2ref_cos))  # Transform points to reference coordinate system
        
        return points_ref[:,0:3]  # Return the transformed points, discarding the homogeneous coordinate


    def ref2Camera(self, points, img_size, bbox_list):
        """Project points from the reference coordinate system onto the camera image plane.
        
        Parameters:
        - points: A numpy array of points in the reference coordinate system.
        - img_size: A tuple representing the size of the camera image (width, height).
        - bbox_list: A list of tuples representing the bounding box coordinates [(x_min, y_min, x_max, y_max), ...].
        
        Returns:
        - A tuple containing a numpy array of the projected points on the camera image plane and a mask
          array indicating which points are within the image frame and inside the bounding box.
        """
        obj_image_list = []
        mask_list = []
        
        for bbox in bbox_list:
            obj_image = np.dot(self.ref2camera[0:3, 0:3], points.T)  # Apply rotation
            T = self.ref2camera[0:3, 3]
            obj_image = obj_image + T[:, np.newaxis]  # Apply translation
            obj_image = np.dot(self.K, obj_image)  # Apply intrinsic camera matrix
            obj_image = obj_image / obj_image[2]  # Normalize by the third row to project onto image plane
            obj_image = np.delete(obj_image, 2, 0)  # Remove the third row

            # Create a mask to filter out points that are outside the image frame or outside the bounding box
            mask = (obj_image[0, :] <= img_size[0]) & \
                   (obj_image[1, :] <= img_size[1]) & \
                   (obj_image[0, :] >= 0) & (obj_image[1, :] >= 0) & \
                   (obj_image[0, :] >= bbox[0]) & (obj_image[0, :] <= bbox[2]) & \
                   (obj_image[1, :] >= bbox[1]) & (obj_image[1, :] <= bbox[3])

            obj_image_list.append(obj_image[:, mask])
            mask_list.append(mask)
        
        return obj_image_list, mask_list  # Return the projected points list and the mask list
    
    def mean_distance(self, points, mask):
        """Calculate the mean Euclidean distance of the points from the camera's viewpoint inside the bounding box.
        
        Parameters:
        - points: A numpy array of points in 3D space.
        - mask: A boolean mask indicating which points are inside the bounding box.
        
        Returns:
        - The mean Euclidean distance of the points from the camera's viewpoint inside the bounding box.
        """
        camera_points = np.dot(self.ref2camera[0:3, 0:3], points) + self.ref2camera[0:3, 3][:, np.newaxis]
        distance = np.linalg.norm(camera_points, axis=0)
        return np.mean(distance)

# Function to calculate fused distance using weighted averaging
def calculate_fused_distance(lidar_mean_distance, radar_mean_distance):
    lidar_weight = 0.7  # Weight for lidar distance
    radar_weight = 0.3  # Weight for radar distance
    return lidar_weight * lidar_mean_distance + radar_weight * radar_mean_distance

# Function to calculate mean absolute error (MAE) and root mean square error (RMSE)
def calculate_accuracy(true_distances, estimated_distances):
    mae = np.nanmean(np.abs(np.array(true_distances) - np.array(estimated_distances)))
    rmse = np.sqrt(np.nanmean((np.array(true_distances) - np.array(estimated_distances))**2))
    return mae, rmse

# Main code
root = r'C:\Users\rahma\Desktop\sensor fusion astyx dataset\dataset_astyx_hires2019\dataset_astyx_hires2019'
input_dir = os.path.join(root, 'Step-1-output')
output_dir = os.path.join(root, 'step-6-output')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        # Extract the image index from the filename
        image_index = filename.split('_')[0]

        # Read the image
        img = cv2.imread(os.path.join(input_dir, filename))

        # Load lidar and radar data
        lidar_data = np.loadtxt(os.path.join(root, 'lidar_vlp16', f"{image_index}.txt"), skiprows=1)
        radar_data = np.loadtxt(os.path.join(root, 'radar_6455', f"{image_index}.txt"), skiprows=2)

        # Load calibration data
        calib = calib_astyx(os.path.join(root, 'calibration', f"{image_index}.json"))

        # Load bounding box coordinates from file
        bbox_file = os.path.join(input_dir, f"{image_index}_vehicle_coordinates.txt")
        bounding_boxes = []
        with open(bbox_file, 'r') as f:
            for line in f:
                bbox_data = line.strip().split()  # Split by spaces
                bbox = tuple(map(int, bbox_data))  # Convert to tuple of integers
                bounding_boxes.append(bbox)

        # Convert lidar points to reference coordinate system
        lidar_data[:, 0:3] = calib.lidar2ref(lidar_data[:, 0:3])

        # Project lidar and radar points onto the camera image plane
        lidarOnImg, lidarOnImgMask = calib.ref2Camera(lidar_data[:, 0:3], (img.shape[1], img.shape[0]), bounding_boxes)
        radarOnImg, radarOnImgMask = calib.ref2Camera(radar_data[:, 0:3], (img.shape[1], img.shape[0]), bounding_boxes)

        # Calculate mean distances
        lidar_points_inside_bbox = [lidar_data[mask] for mask in lidarOnImgMask]
        lidar_mean_distance = [calib.mean_distance(points[:, 0:3].T, np.ones(len(points), dtype=bool)) for points in lidar_points_inside_bbox]

        radar_points_inside_bbox = [radar_data[mask] for mask in radarOnImgMask]
        radar_mean_distance = [calib.mean_distance(points[:, 0:3].T, np.ones(len(points), dtype=bool)) for points in radar_points_inside_bbox]

        true_distances_file = os.path.join(root, 'groundtruth_obj3d', f"{image_index}.json")
        with open(true_distances_file, 'r') as f:
            true_distances_data = json.load(f)
        true_distances = []

        for obj in true_distances_data['objects']:
            center3d = np.array(obj['center3d'])
            true_distances.append(np.linalg.norm(center3d))  # Calculate Euclidean distance for each center3d coordinate

        mae_list = []
        rmse_list = []

        for lidar_points, radar_points, lidar_distance, radar_distance, true_distance, bbox in zip(lidar_points_inside_bbox, radar_points_inside_bbox, lidar_mean_distance, radar_mean_distance, true_distances, bounding_boxes):
            fused_distance = calculate_fused_distance(lidar_distance, radar_distance)
            estimated_distances = [fused_distance] * len(lidar_points)  # Use the same fused distance for all lidar and radar points inside the same bbox
            mae, rmse = calculate_accuracy([true_distance] * len(lidar_points), estimated_distances)
            mae_list.append(mae)
            rmse_list.append(rmse)

            # Add text annotation for MAE and RMSE on top of each bounding box
            bbox_center_x = (bbox[0] + bbox[2]) / 2
            bbox_center_y = bbox[1] - 10  # Slightly above the bounding box
            cv2.putText(img, f'MAE: {mae:.2f}, RMSE: {rmse:.2f}', (int(bbox_center_x), int(bbox_center_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Save annotated image
        output_filename = os.path.join(output_dir, f"{image_index}_vehicle_detection_annotated.jpg")
        cv2.imwrite(output_filename, img)



C:\Users\rahma\AppData\Local\Temp\ipykernel_30156\1040005436.py:133: RuntimeWarning: Mean of empty slice
  mae = np.nanmean(np.abs(np.array(true_distances) - np.array(estimated_distances)))
C:\Users\rahma\AppData\Local\Temp\ipykernel_30156\1040005436.py:134: RuntimeWarning: Mean of empty slice
  rmse = np.sqrt(np.nanmean((np.array(true_distances) - np.array(estimated_distances))**2))
